**Python code**

In [ ]:
# Step 1: Install Required Libraries
!pip install pandas openpyxl

# Step 2: Upload an Excel File to Google Colab
from google.colab import files

print("Please upload your Excel file:")
uploaded = files.upload()  # Prompt the user to upload the Excel file

# Step 3: Read and Process the Excel File
import pandas as pd

# Get the uploaded file name
file_name = list(uploaded.keys())[0]

try:
    # Step 4: Load the Excel file into a pandas DataFrame
    df = pd.read_excel(file_name)
    print(f"Data loaded successfully! Here are the first few rows:\n{df.head()}")

    # Step 5: Ensure the First Column is 'Name'
    first_column = df.columns[0]
    if first_column.lower() != 'name':
        raise ValueError(f"The first column is not 'Name'. Found '{first_column}' instead.")

    # Step 6: Calculate Metadata Summary
    name_non_null_count = df['Name'].notnull().sum()
    if name_non_null_count == 0:
        raise ValueError("The 'Name' column has no non-null values, so percentages cannot be calculated.")

    summary = []
    for column in df.columns:
        non_null_count = df[column].notnull().sum()

        # Count occurrences of 'bkp', 'bck', 'bk', or 'backup' (case-insensitive) within each entry, including substrings
        backup_count = df[column].astype(str).str.contains(r'(bkp|bck|bk|backup)', case=False, na=False).sum()

        percentage_of_name = (non_null_count / name_non_null_count * 100)

        summary.append({
            'Column Name': column,
            'MetaData Counts': non_null_count,
            'Percentage Of Completion': f"{percentage_of_name:.2f}%",
            'Backups Count': backup_count,
        })

    # Create a DataFrame for the summary
    summary_df = pd.DataFrame(summary)
    print("\nMetadata Summary:")
    print(summary_df)

    # Step 7: Save the Summary to a New Excel File
    output_file = "metadata_summary_with_backups.xlsx"
    summary_df.to_excel(output_file, index=False)
    print(f"\nMetadata summary saved as {output_file}.")

    # Step 8: Download the Processed File
    files.download(output_file)

except Exception as e:
    print(f"Error processing the file: {e}")


**SAS** **Code**



In [ ]:
# Install SASPy
!pip install saspy

# Import SASPy
import saspy
import pandas as pd

# Establish a connection to a remote or local SAS environment
sas = saspy.SASsession(cfgname='iom')  # Use 'sas' for local SAS or 'iom' for remote SAS server

# SAS Code for Processing Excel Files
sas_code = """
    libname mylib 'path-to-your-directory';  /* Define your SAS library location */

    /* Import the Excel file */
    proc import datafile='path-to-your-file.xlsx'
        out=mylib.mydata
        dbms=xlsx
        replace;
    run;

    /* Generate Metadata Summary */
    proc contents data=mylib.mydata out=work.metadata;
    run;

    data work.metadata_summary;
        set work.metadata;
        backup_count = 0;
        if index(upcase(name), 'BKP') > 0 or index(upcase(name), 'BCK') > 0 or
           index(upcase(name), 'BK') > 0 or index(upcase(name), 'BACKUP') > 0 then backup_count = 1;
    run;

    proc print data=work.metadata_summary;
    run;
"""

# Execute the SAS code
sas.submit(sas_code)
